In [ ]:
# Install tf libraries
!pip3 install bert-tensorflow==1.0.1 tensorflow==1.15

     |████████████████████████████████| 71kB 10.0MB/s 
     |████████████████████████████████| 412.3MB 38kB/s 
     |████████████████████████████████| 3.8MB 51.3MB/s 
     |████████████████████████████████| 512kB 50.7MB/s 
     |████████████████████████████████| 51kB 8.3MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=2746d216d1ed429e21789d29ed944fb61954ef267fd87e6ee52c8e456d569404
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorfl

In [ ]:
# Download pre-trained model BERT-Base
!wget https://f000.backblazeb2.com/file/malaya-model/bert-bahasa/bert-base-2020-10-08.tar.gz
!wget https://raw.githubusercontent.com/huseinzol05/Malaya/master/pretrained-model/bert/BERT.wordpiece
!wget https://raw.githubusercontent.com/huseinzol05/Malaya/master/pretrained-model/bert/config/BASE_config.json
!tar -zxf bert-base-2020-10-08.tar.gz
!ls

--2021-02-27 06:23:01--  https://f000.backblazeb2.com/file/malaya-model/bert-bahasa/bert-base-2020-10-08.tar.gz
Resolving f000.backblazeb2.com (f000.backblazeb2.com)... 104.153.233.177
Connecting to f000.backblazeb2.com (f000.backblazeb2.com)|104.153.233.177|:443... connected.
HTTP request sent, awaiting response... 200 
Length: 1217112118 (1.1G) [application/octet-stream]
Saving to: ‘bert-base-2020-10-08.tar.gz’

bert-base-2020-10-0 100%[===================>]   1.13G  29.3MB/s    in 44s     

2021-02-27 06:23:46 (26.5 MB/s) - ‘bert-base-2020-10-08.tar.gz’ saved [1217112118/1217112118]

--2021-02-27 06:23:46--  https://raw.githubusercontent.com/huseinzol05/Malaya/master/pretrained-model/bert/BERT.wordpiece
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2241

In [ ]:
!ls bert-base

model.ckpt-1000000.data-00000-of-00001	model.ckpt-1000000.meta
model.ckpt-1000000.index


In [ ]:
!pip3 install utils
import sys

sys.path.insert(0, '../')
import utils

In [ ]:
# Load labelled sentiment data
import pandas as pd

df = pd.read_csv('sentiment-data-v2.csv')
df.tail()

,label,text
3680,Positive,Jelas pembangkang buat tuduhan untuk mengeliru...
3681,Positive,demokrasi adalah kuasa rakyat di mana pegawai ...
3682,Positive,"Selain dapat menyelesaikan isu beg berat, peng..."
3683,Positive,Hospital Langkawi buat masa ini hanya dapat me...
3684,Positive,Jika sebelum ini kita selesa bergerak dalam ‘g...


In [ ]:
# Convert df to list
labels = df['label'].values.tolist()
texts = df['text'].values.tolist()
unique_labels = sorted(list(set(labels)))
unique_labels

['Negative', 'Positive']

In [ ]:
import tensorflow as tf
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization
from bert import modeling

In [ ]:
tokenizer = tokenization.FullTokenizer(vocab_file = 'BERT.wordpiece', do_lower_case = False)
tokens = tokenizer.tokenize('Asyraf hensem sangat')
tokens

['Asyraf', 'he', '##n', '##sem', 'sangat']

In [ ]:
tokenizer.convert_tokens_to_ids(tokens)

[22589, 3788, 1015, 29643, 2521]

In [ ]:
def token_to_ids(text, maxlen = 512):
    tokens_a = tokenizer.tokenize(text)
    if len(tokens_a) > maxlen - 2:
        tokens_a = tokens_a[:(maxlen - 2)]
    tokens = ['[CLS]'] + tokens_a + ['[SEP]']
    segment_id = [0] * len(tokens)
    input_mask = [1] * len(tokens)
    input_id = tokenizer.convert_tokens_to_ids(tokens)
    return {'tokens': tokens, 'input_id': input_id,
    'input_mask': input_mask, 'segment_id': segment_id}

In [ ]:
# Tokenize text 
token_to_ids(texts[0])

{'input_id': [2,
  4015,
  17,
  2009,
  2088,
  1822,
  5714,
  6332,
  1766,
  3062,
  3558,
  16,
  20153,
  1828,
  3718,
  2766,
  20018,
  18,
  3],
 'input_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'segment_id': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'tokens': ['[CLS]',
  'Lebih',
  '-',
  'lebih',
  'lagi',
  'dengan',
  'kemudahan',
  'internet',
  'dan',
  'laman',
  'sosial',
  ',',
  'taktik',
  'ini',
  'semakin',
  'mudah',
  'dikembangkan',
  '.',
  '[SEP]']}

In [ ]:
# Data pipeline
def generate():
    while True:
        for i in range(len(texts)):
            if len(texts[i]) > 5:
                d = token_to_ids(texts[i])
                d['label'] = [unique_labels.index(labels[i])]
                d.pop('tokens', None)
                yield d

In [ ]:
g = generate()
next(g)

{'input_id': [2,
  4015,
  17,
  2009,
  2088,
  1822,
  5714,
  6332,
  1766,
  3062,
  3558,
  16,
  20153,
  1828,
  3718,
  2766,
  20018,
  18,
  3],
 'input_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'label': [0],
 'segment_id': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

In [ ]:
def get_dataset(batch_size = 32, shuffle_size = 128):
  # Shuffle_size 32 > 128 - 27/2/2021
    def get():
        dataset = tf.data.Dataset.from_generator(
            generate,
            {'input_id': tf.int32, 'input_mask': tf.int32, 'segment_id': tf.int32, 'label': tf.int32},
            output_shapes = {
                'input_id': tf.TensorShape([None]),
                'input_mask': tf.TensorShape([None]),
                'segment_id': tf.TensorShape([None]),
                'label': tf.TensorShape([None])
            },
        )
        dataset = dataset.shuffle(shuffle_size)
        dataset = dataset.padded_batch(
            batch_size,
            padded_shapes = {
                'input_id': tf.TensorShape([None]),
                'input_mask': tf.TensorShape([None]),
                'segment_id': tf.TensorShape([None]),
                'label': tf.TensorShape([None])
            },
            padding_values = {
                'input_id': tf.constant(0, dtype = tf.int32),
                'input_mask': tf.constant(0, dtype = tf.int32),
                'segment_id': tf.constant(0, dtype = tf.int32),
                'label': tf.constant(0, dtype = tf.int32),
            },
        )
        return dataset
    return get

In [ ]:
# Test data pipeline using tf.session
tf.reset_default_graph()
sess = tf.InteractiveSession()
iterator = get_dataset()()
iterator = iterator.make_one_shot_iterator().get_next()

/usr/local/lib/python3.7/dist-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [ ]:
iterator

{'input_id': <tf.Tensor 'IteratorGetNext:0' shape=(?, ?) dtype=int32>,
 'input_mask': <tf.Tensor 'IteratorGetNext:1' shape=(?, ?) dtype=int32>,
 'label': <tf.Tensor 'IteratorGetNext:2' shape=(?, ?) dtype=int32>,
 'segment_id': <tf.Tensor 'IteratorGetNext:3' shape=(?, ?) dtype=int32>}

In [ ]:
sess.run(iterator)

{'input_id': array([[    2,  7824,  5119, ...,     0,     0,     0],
        [    2,  2616, 11334, ...,     0,     0,     0],
        [    2,  2508, 15201, ...,     0,     0,     0],
        ...,
        [    2,  3130,  1874, ...,     0,     0,     0],
        [    2,  6542,  1011, ...,     0,     0,     0],
        [    2,  3130,  7140, ...,     0,     0,     0]], dtype=int32),
 'input_mask': array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]], dtype=int32),
 'label': array([[1],
        [1],
        [1],
        [0],
        [1],
        [1],
        [0],
        [1],
        [1],
        [1],
        [1],
        [0],
        [0],
        [1],
        [1],
        [1],
        [0],
        [0],
        [1],
        [1],
        [0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [1],
        [1],
     

In [ ]:
# Define BERT model
bert_config = modeling.BertConfig.from_json_file('BASE_config.json')
bert_config.__dict__

{'attention_probs_dropout_prob': 0.1,
 'directionality': 'bidi',
 'hidden_act': 'gelu',
 'hidden_dropout_prob': 0.1,
 'hidden_size': 768,
 'initializer_range': 0.02,
 'intermediate_size': 3072,
 'max_position_embeddings': 512,
 'num_attention_heads': 12,
 'num_hidden_layers': 12,
 'pooler_fc_size': 768,
 'pooler_num_attention_heads': 12,
 'pooler_num_fc_layers': 3,
 'pooler_size_per_head': 128,
 'pooler_type': 'first_token_transform',
 'type_vocab_size': 2,
 'vocab_size': 32000}

In [ ]:
epoch = 30
warmup_proportion = 0.1
num_warmup_steps = int(epoch * warmup_proportion)
learning_rate = 2e-5
init_checkpoint = 'bert-base/model.ckpt-1000000'

In [ ]:
# Model_fn accept 4 parameters, define training loss and accuracy
import utils_lib
def model_fn(features, labels, mode, params):
    Y = tf.cast(features['label'][:, 0], tf.int32)

    model = modeling.BertModel(
        config = bert_config,
        is_training = True,
        input_ids = features['input_id'],
        input_mask = features['input_mask'],
        token_type_ids = features['segment_id'],
        # use_one_hot_embeddings = False,
        use_one_hot_embeddings = True,
    )
    output_layer = model.get_pooled_output()
    logits = tf.layers.dense(output_layer, 2)
    loss = tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits = logits, labels = Y
        )
    )

    tf.identity(loss, 'train_loss')

    accuracy = tf.metrics.accuracy(
        labels = Y, predictions = tf.argmax(logits, axis = 1)
    )
    tf.identity(accuracy[1], name = 'train_accuracy')

    variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)

    assignment_map, initialized_variable_names = utils_lib.get_assignment_map_from_checkpoint(
        variables, init_checkpoint
    )

    tf.train.init_from_checkpoint(init_checkpoint, assignment_map)

    if mode == tf.estimator.ModeKeys.TRAIN:
        train_op = optimization.create_optimizer(loss, learning_rate, epoch, num_warmup_steps, False)
        estimator_spec = tf.estimator.EstimatorSpec(
            mode = mode, loss = loss, train_op = train_op
        )

    elif mode == tf.estimator.ModeKeys.EVAL:
        estimator_spec = tf.estimator.EstimatorSpec(
            mode = tf.estimator.ModeKeys.EVAL,
            loss = loss,
            eval_metric_ops = {'accuracy': accuracy},
        )

    return estimator_spec

In [ ]:
# Initiate training session
train_dataset = get_dataset()

In [ ]:
# from google.colab import files
# src = list(files.upload().values())[0]
# open('__init__.py','wb').write(src)
# import utils

In [ ]:
# !pip3 install utils
# Reminder: Add utils_lib.py before training. ada dalam folder master doc

# from utils_lib import utils_lib
import utils_lib

train_hooks = [
    tf.train.LoggingTensorHook(
        ['train_accuracy', 'train_loss'], every_n_iter = 1
    )
]

utils_lib.run_training(
    train_fn = train_dataset,
    model_fn = model_fn,
    model_dir = 'finetuned-bert-base',
    num_gpus = 1,
    log_step = 1,
    save_checkpoint_step = epoch,
    max_steps = epoch,
    train_hooks = train_hooks,
)

INFO:tensorflow:Using config: {'_model_dir': 'finetuned-bert-base', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 30, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 1, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f5727f24450>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Skipping training since max_steps has already saved.


In [ ]:
# TODO: Use different datasets, Calculate Precision, F1-score, etc.